In [41]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn import metrics
import nltk
from nltk import word_tokenize
from nltk.corpus import reuters
import sys
sys.path.append('../code/')
import wk
from sklearn.metrics import accuracy_score
import plotly.plotly as py
from plotly.tools import FigureFactory as FF 

EARN = 0
ACQ = 1
CRUDE = 2
CORN = 3

In [43]:
#import data

print "importing train + test data..."
all_files = reuters.fileids()
#print(len(all_files))

train_files = list(filter(lambda file: file.startswith('train'), all_files))
#print(len(train_files))

test_files = list(filter(lambda file: file.startswith('test'), all_files))
#print(len(test_files))

categories = reuters.categories()

earn_docs = reuters.fileids('earn')
acq_docs = reuters.fileids('acq')
crude_docs = reuters.fileids('crude')
corn_docs = reuters.fileids('corn')

earn_train_docs = [w for w in earn_docs if w in train_files]   
earn_test_docs = [w for w in earn_docs if w in test_files]  

acq_train_docs = [w for w in acq_docs if w in train_files]   
acq_test_docs = [w for w in acq_docs if w in test_files]  

crude_train_docs = [w for w in crude_docs if w in train_files]   
crude_test_docs = [w for w in crude_docs if w in test_files]  

corn_train_docs = [w for w in corn_docs if w in train_files]   
corn_test_docs = [w for w in corn_docs if w in test_files]  

TrainDocVals = []
TrainDocLabels = []
TestDocVals = []
TestDocLabels = []

for docHandle in earn_train_docs[0:10]:
    TrainDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TrainDocLabels.append(EARN)
    
for docHandle in acq_train_docs[0:10]:
    TrainDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TrainDocLabels.append(ACQ)
    
for docHandle in crude_train_docs[0:10]:
    TrainDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TrainDocLabels.append(CRUDE)
    
for docHandle in corn_train_docs[0:10]:
    TrainDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TrainDocLabels.append(CORN)
    
for docHandle in earn_test_docs[0:10]:
    TestDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TestDocLabels.append(EARN)
    
for docHandle in acq_test_docs[0:10]:
    TestDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TestDocLabels.append(ACQ)
    
for docHandle in crude_test_docs[0:10]:
    TestDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TestDocLabels.append(CRUDE)
    
for docHandle in corn_test_docs[0:10]:
    TestDocVals.append(" ".join(word_tokenize(reuters.raw(docHandle))))
    TestDocLabels.append(CORN)
    
print "done"
    

importing train + test data...
done


In [44]:
print "computing WK gram matrix"
WKTestGram = np.ones((len(TestDocVals),len(TestDocVals)))
WKTrainGram = np.ones((len(TrainDocVals),len(TrainDocVals)))
for i in xrange( 0, len(TrainDocVals) ):
    if( (i+1)%10 == 0 ):
        print "row %d of %d\n" % ( i+1, len(TrainDocVals) )       
    for j in xrange(0,len(TrainDocVals)):
        WKTrainGram[i][j] = wk.wk(TrainDocVals[i], TrainDocVals[j])
        
for i in xrange( 0, len(TestDocVals) ):
    if( (i+1)%10 == 0 ):
        print "row %d of %d\n" % ( i+1, len(TestDocVals) )       
    for j in xrange(0,len(TestDocVals)):
        WKTestGram[i][j] = wk.wk(TestDocVals[i], TestDocVals[j])
        
print "done"

computing WK gram matrix
row 10 of 40

row 20 of 40

row 30 of 40

row 40 of 40

row 10 of 40

row 20 of 40

row 30 of 40

row 40 of 40

done


In [50]:
clf = svm.SVC(kernel='precomputed')

clf.fit(WKTrainGram, TrainDocLabels)
label_pred = clf.predict(WKTestGram)


 
precision, recall, fscore, support = metrics.precision_recall_fscore_support(TestDocLabels, label_pred)

print "Precision:"
print precision
print "recall:" 
print recall
print "F1: "
print fscore

#print_results_table(precision,recall,fscore)

data_matrix = [['Label', 'F1 Score', 'Precision', 'Recall'],
                   ['Earn', fscore[EARN],precision[EARN],recall[EARN]],
                   ['ACQ', fscore[ACQ],precision[ACQ],recall[ACQ]],
                   ['CRUDE', fscore[CRUDE],precision[CRUDE],recall[CRUDE]],
                   ['CORN', fscore[CORN],precision[CORN],recall[CORN]]]

table = FF.create_table(data_matrix)
py.iplot(table, filename='F1 Results')

Precision:
[ 1.          0.83333333  1.          1.        ]
recall:
[ 0.9  1.   0.9  1. ]
F1: 
[ 0.94736842  0.90909091  0.94736842  1.        ]
